# INTRODUCTION
In this notebook we will be using the following packages as bullet list:
  * [pyspark](https://spark.apache.org/docs/latest/api/python/pyspark.html)
  * [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  * [pyarrow](https://arrow.apache.org/docs/python/index.html)

We will be using the following data:
  * dressipi_recsys2022

The dataset is split in 4 csv files :
   * candidate_items : contains the candidate items for each user
   * item_features : contains the features for each item (items can have multiple features)
   * train_purchases : contains the purchases for each user
   * train_sessions : contains the sessions for each user



In [2]:
# Uncomment to install the required packages
# %pip install pyspark
# %pip install pyarrow


  Using cached pyarrow-8.0.0-cp39-cp39-win_amd64.whl (17.9 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\natha\AppData\Local\Programs\Python\Python39\UndCliMo\Scripts\python.exe -m pip install --upgrade pip' command.


In [49]:
from dataset import *
from pipeline import *
from our_model import *
from pyspark.sql import SparkSession

# start spark session
spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

candidate_items, item_features, train_purchases, train_sessions = [None] * 4
datasets = [candidate_items, item_features, train_purchases, train_sessions]

#load all datasets
def load_candidate_items():
    global candidate_items
    candidate_items = SPARK.read.csv("dressipi_recsys2022/candidate_items.csv", header=True)
    candidate_items = candidate_items.withColumn("item_id", candidate_items["item_id"].cast("int"))

def load_item_features():
    global item_features
    item_features = SPARK.read.csv("dressipi_recsys2022/item_features.csv", header=True)
    item_features = item_features.withColumn("item_id", item_features["item_id"].cast("int"))
    item_features = item_features.withColumn("feature_category_id", item_features["feature_category_id"].cast("int"))
    item_features = item_features.withColumn("feature_value_id", item_features["feature_value_id"].cast("int"))

def load_train_purchases():
    global train_purchases
    train_purchases = SPARK.read.csv("dressipi_recsys2022/train_purchases.csv", header=True)
    train_purchases = train_purchases.withColumn("session_id", train_purchases["session_id"].cast("int"))
    train_purchases = train_purchases.withColumn("item_id", train_purchases["item_id"].cast("int"))
    train_purchases = train_purchases.withColumn("date", train_purchases["date"].cast("timestamp"))

def load_train_sessions():
    global train_sessions
    train_sessions = SPARK.read.csv("dressipi_recsys2022/train_sessions.csv", header=True)
    train_sessions = train_sessions.withColumn("session_id", train_sessions["session_id"].cast("int"))
    train_sessions = train_sessions.withColumn("user_id", train_sessions["item_id"].cast("int"))
    train_sessions = train_sessions.withColumn("date", train_sessions["date"].cast("timestamp"))

def load_datasets():
    global datasets
    load_candidate_items()
    load_item_features()
    load_train_purchases()
    load_train_sessions()
    datasets = [candidate_items, item_features, train_purchases, train_sessions]

load_datasets()

# are there missing values (na or null) in any of those dataframes ? print the number of missing values
print("candidate_items:", candidate_items.filter(candidate_items["item_id"].isNull()).count())
print("item_features:", item_features.filter(item_features["item_id"].isNull()).count())
print("train_purchases:", train_purchases.filter(train_purchases["item_id"].isNull()).count())
print("train_sessions:", train_sessions.filter(train_sessions["item_id"].isNull()).count())


# show the first elements of the dataset
train_sessions.show(5)

candidate_items: 0
item_features: 0
train_purchases: 0
train_sessions: 0
+----------+-------+--------------------+-------+
|session_id|item_id|                date|user_id|
+----------+-------+--------------------+-------+
|         3|   9655|2020-12-18 21:25:...|   9655|
|         3|   9655|2020-12-18 21:19:...|   9655|
|        13|  15654|2020-03-13 19:35:...|  15654|
|        18|  18316|2020-08-26 19:18:...|  18316|
|        18|   2507|2020-08-26 19:16:...|   2507|
+----------+-------+--------------------+-------+
only showing top 5 rows



In [38]:
item_features.show(5)

+-------+-------------------+----------------+
|item_id|feature_category_id|feature_value_id|
+-------+-------------------+----------------+
|      2|                 56|             365|
|      2|                 62|             801|
|      2|                 68|             351|
|      2|                 33|             802|
|      2|                 72|              75|
+-------+-------------------+----------------+
only showing top 5 rows



In [50]:
# show rows with item_id == 2 in all dataframes
for d in datasets:
    print(d.filter(d["item_id"] == 2).collect())

[]
[Row(item_id=2, feature_category_id=56, feature_value_id=365), Row(item_id=2, feature_category_id=62, feature_value_id=801), Row(item_id=2, feature_category_id=68, feature_value_id=351), Row(item_id=2, feature_category_id=33, feature_value_id=802), Row(item_id=2, feature_category_id=72, feature_value_id=75), Row(item_id=2, feature_category_id=29, feature_value_id=123), Row(item_id=2, feature_category_id=16, feature_value_id=38), Row(item_id=2, feature_category_id=50, feature_value_id=76), Row(item_id=2, feature_category_id=61, feature_value_id=462), Row(item_id=2, feature_category_id=53, feature_value_id=6), Row(item_id=2, feature_category_id=7, feature_value_id=394), Row(item_id=2, feature_category_id=69, feature_value_id=885), Row(item_id=2, feature_category_id=47, feature_value_id=123)]
[]
[Row(session_id=3401213, item_id='2', date=datetime.datetime(2021, 3, 28, 8, 24, 15, 321000), user_id=2)]
